In [1]:
import pandas as pd
import numpy as np

In [2]:
members_13 = pd.read_csv('members_13.csv', parse_dates=['birthday', 'entry_date'])
members_13.head(3)

member_name   birthday sex                       profession  salary_usd$  \
0  Vanessa Casey 1960-08-08   F  Engineer, manufacturing systems         1884   
1   Cheryl Carey 1958-12-11   N     Trade union research officer         1518   
2  Kendra Valdez 1962-06-16   M               Press photographer         1669   

  entry_date  group_id   city_name     state_name  age  \
0 2018-05-24  19927387  Montgomery  West Virginia   63   
1 2019-11-14  19712846     Colonia     New Jersey   65   
2 2018-08-06  10528562    Pinckney       Michigan   61   

                  fun_club_name  category_id  monthly_fee_USD$  \
0  Events That Make You Smarter            6                 7   
1     Mix Professionals Chicago            9                18   
2  Madison Square Park Mommies!           25                19   

          category_name  membership_duration_days  sum_fee_paid, USD$  
0  Education & Learning                      1803               421.0  
1               Fitness                      1264               758.0  
2      Parents & Family                      1729              1095.0

In [3]:
# переименуем "проблемные колонки"
members_13 = members_13.rename(columns={'sex':'member_gender', 'age':'member_age','birthday':'member_birthday','salary_usd$':'member_salary', 'monthly_fee_USD$':'monthly_fee', 'sum_fee_paid, USD$':'sum_fee_paid'})

In [4]:
founders_1=pd.read_csv('founders_1.csv', parse_dates=['birthday', 'foundation_date'])
founders_1.head(3)

name   birthday sex             email  \
0     Daniel Carter 1965-01-12   M     law@gmail.com   
1    Samuel Ramirez 1995-12-06   N     pay@gmail.com   
2  Frederick Keller 1960-05-10   F  factor@gmail.com   

                                             address  \
0  0172 Howard Hill Suite 460\nNorth Peterborough...   
1  41969 Seth Centers Apt. 977\nBakerville, OK 04560   
2                   Unit 4661 Box 5647\nDPO AP 93978   

                      company          profession  salary_usd$  \
0  Campbell, Wang and Russell  Furniture designer         4889   
1  Ross, Wallace and Williams            Best boy         4109   
2    Ward, Adams and Gonzalez  Editor, film/video         4461   

  foundation_date  city_id  city_name  state_id state_code     state_name  \
0      2017-12-26   121610     Medway      1433         MA  Massachusetts   
1      2018-07-08   125110   Richwood      1417         NJ     New Jersey   
2      2019-08-05   124330  Plainview      1452         NY       New York   

                 funs_club  age  
0   Alternative Health NYC   58  
1       Energy Healers NYC   28  
2  Flourishing Life Meetup   63

In [5]:
founders_1 = founders_1.rename(columns={'sex':'founder_gender','name':'founder_name', 'age':'founder_age','birthday':'founder_birthday','salary_usd$':'founder_salary'})

# Задача: вычислить какую сумму членских взносов собирает каждый владелец клуба ежемесячно

In [6]:
# некоторые основатели ведут несколько клубов одновременно
founders_1[['funs_club', 'founder_name']].nunique()

funs_club       8034
founder_name    7605
dtype: int64

In [7]:
# выведем фрейм со списком мульти-владельцев
df=founders_1.groupby('founder_name')['funs_club'].count().to_frame()
multi_founders=df[df['funs_club'] > 1].reset_index()
multi_founders.sort_values(by='funs_club', ascending=False)

founder_name  funs_club
146  Jennifer Smith          8
122     James Smith          6
357   William Smith          5
124  James Williams          5
351   William Brown          5
..              ...        ...
125    James Wilson          2
123  James Townsend          2
121   James Simpson          2
120      James Ruiz          2
360   Zachary Lewis          2

[361 rows x 2 columns]

In [8]:
founders_1.head(1)

founder_name founder_birthday founder_gender          email  \
0  Daniel Carter       1965-01-12              M  law@gmail.com   

                                             address  \
0  0172 Howard Hill Suite 460\nNorth Peterborough...   

                      company          profession  founder_salary  \
0  Campbell, Wang and Russell  Furniture designer            4889   

  foundation_date  city_id city_name  state_id state_code     state_name  \
0      2017-12-26   121610    Medway      1433         MA  Massachusetts   

                funs_club  founder_age  
0  Alternative Health NYC           58

In [9]:
members_13.head(1)

member_name member_birthday member_gender  \
0  Vanessa Casey      1960-08-08             F   

                        profession  member_salary entry_date  group_id  \
0  Engineer, manufacturing systems           1884 2018-05-24  19927387   

    city_name     state_name  member_age                 fun_club_name  \
0  Montgomery  West Virginia          63  Events That Make You Smarter   

   category_id  monthly_fee         category_name  membership_duration_days  \
0            6            7  Education & Learning                      1803   

   sum_fee_paid  
0         421.0

In [10]:
# значения зарплат основателей надодятся в таблице founder, а значения ежемесячных ставок членских взносов в таблице members
# поэтому объединим таблицы
founders_2=founders_1[['founder_name','funs_club', 'founder_salary']].\
           merge(members_13[['member_name', 'fun_club_name', 'monthly_fee', 'sum_fee_paid']],
           how='left', left_on='funs_club', right_on='fun_club_name')
# суммы взносов собранные владельцами ежемесячно (сортировка по убыванию)
founders_2.groupby('founder_name')['monthly_fee'].sum().reset_index().sort_values('monthly_fee', ascending=False)

founder_name  monthly_fee
3259   Jennifer Smith         1886
7497    William Brown         1356
3179   Jennifer Brown         1331
2944      James Smith         1170
6239     Robert Smith         1107
...               ...          ...
1304    Charles Sloan            0
1319     Chelsea King            0
3780   Joshua Chapman            0
1334    Cheryl Keller            0
6078  Regina Gonzales            0

[7605 rows x 2 columns]

In [11]:
# соотношение суммарной зарплаты и суммы от поступления взносов (сортировка по убыванию суммы взносов)
founders_2.groupby('founder_name', as_index=False).agg({'sum_fee_paid':'sum', 'founder_salary':'sum'})\
                    .sort_values(by='sum_fee_paid', ascending=False)

founder_name  sum_fee_paid  founder_salary
3259     Jennifer Smith      105608.0          473598
7497      William Brown       73217.0          265476
3179     Jennifer Brown       69786.0          213984
6239       Robert Smith       62650.0          196923
2944        James Smith       61413.0          265688
...                 ...           ...             ...
4855        Maria Grant           0.0           52948
7031  Teresa Fitzgerald           0.0           41648
1180        Carol Hayes           0.0           33456
1189      Carol Watkins           0.0           31738
6695        Sheila Horn           0.0           53042

[7605 rows x 3 columns]

In [12]:
# посчитаем, какой процент от месячной зарплаты основателей составляют собираемые клубные взносы
founders_month_toll=founders_2.groupby('founder_name', as_index=False)['monthly_fee'].sum()
founders_month_toll

founder_name  monthly_fee
0          Aaron Barr           70
1         Aaron Black          228
2         Aaron Burke          459
3         Aaron Craig           20
4        Aaron Curtis          300
...               ...          ...
7600  Zachary Shelton           13
7601    Zachary Simon          345
7602    Zachary Terry          209
7603    Zachary Yoder          400
7604     Zoe Ferguson          112

[7605 rows x 2 columns]

In [13]:
perc_fee_monthly=founders_2[['founder_name', 'founder_salary']].drop_duplicates().merge(founders_month_toll, on='founder_name')
perc_fee_monthly

founder_name  founder_salary  monthly_fee
0        Daniel Carter            4889          361
1        Daniel Carter            1770          361
2       Samuel Ramirez            4109          133
3     Frederick Keller            4461          160
4        Lauren Garcia            3194          208
...                ...             ...          ...
8029      James Snyder            2848           91
8030        Wayne Ross            2242          240
8031     John Alvarado            2617          273
8032  Russell Calderon            4558          437
8033      Angela Davis            3183          276

[8034 rows x 3 columns]

In [14]:
# выведем топ-10 владельцев клубов по проценту ежемесячных поступлений членских взносов
perc_fee_monthly['perc_fee_monthly']=(perc_fee_monthly['monthly_fee']/perc_fee_monthly['founder_salary']*100).round(1)
perc_fee_monthly.nlargest(10, 'perc_fee_monthly')

founder_name  founder_salary  monthly_fee  perc_fee_monthly
2300    Jennifer Smith            2120         1886              89.0
4196    Jennifer Brown            1572         1331              84.7
2306    Jennifer Smith            2315         1886              81.5
836      William Brown            2001         1356              67.8
834      William Brown            2144         1356              63.2
4197    Jennifer Brown            2133         1331              62.4
6607  Alexander Moreno            1600          988              61.8
2303    Jennifer Smith            3074         1886              61.4
468        James Smith            2013         1170              58.1
2301    Jennifer Smith            3395         1886              55.6